这一个教程主要以一个词袋的模型来介绍下如何用 pytorch 搭建 Network Components。详细信息可以参考[官方文档](https://pytorch.org/tutorials/beginner/nlp/deep_learning_tutorial.html)。官方文档罗列了一些有关深度学习的基础知识以及词袋模型的简介。

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [23]:
data = [("me gusta comer en la cafeteria".split(), "SPANISH"),
        ("Give it to me".split(), "ENGLISH"),
        ("No creo que sea una buena idea".split(), "SPANISH"),
        ("No it is not a good idea to get lost at sea".split(), "ENGLISH")]

test_data = [("Yo creo que si".split(), "SPANISH"),
             ("it is lost on me".split(), "ENGLISH")]

# word_to_ix maps each word in the vocab to a unique integer, which will be its
# index into the Bag of words vector
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

label_to_ix = {'SPANISH': 0, 'ENGLISH': 1}
ix_to_label = {0: 'SPANISH', 1: 'ENGLISH'}

class BowClassifier(nn.Module):
    def __init__(self, vocab_size, num_labels):
        super(BowClassifier, self).__init__()
        self.linear = nn.Linear(vocab_size, num_labels)
    
    def forward(self, input_word_vec):
        output = self.linear(input_word_vec)
        pred = F.log_softmax(output, dim=1)
        return pred

def make_bow_vector(sentence, word_to_ix):
    word_vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        word_vec[word_to_ix[word]] += 1
    return word_vec.view(1, -1)

def make_label_vector(label, label_to_ix):
    label_vec = torch.LongTensor([label_to_ix[label]])
    return label_vec

model = BowClassifier(VOCAB_SIZE, NUM_LABELS)

# we can uncomment to see the size of model prarameters
# the first is A, the second is bias
# for param in model.parameters():
#     print(param.size())

# we can feed a sample data to model to see its output.
# uncomment to see the result
# with torch.no_grad():
#     sample = data[0]
#     bow_vector = make_bow_vector(sample[0], word_to_ix)
#     log_probs = model(bow_vector)
#     print(log_probs)

# Run on test data before we train, just to see a before-and-after
print("Before training: ")
with torch.no_grad():
    for instance, label in test_data:
        bow_vec = make_bow_vector(instance, word_to_ix)
        log_pred = model(bow_vec)
        print("The log_pred is : ", log_pred)
        ix = torch.max(log_pred, dim=1)[1]
        print("The label is: ", ix_to_label[ix.item()])

print("-" * 10)
print("Training...")

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(100):
    for sentence, label in data:
        bow_vec = make_bow_vector(sentence, word_to_ix)
        label_vec = make_label_vector(label, label_to_ix)
        log_pred = model(bow_vec)
        loss = loss_function(log_pred, label_vec)
        print("epoch: {}, loss: {}".format(epoch, loss))
        
        model.zero_grad()
        loss.backward()
        optimizer.step()

print("-" * 10)
print("After training: ")
with torch.no_grad():
    for instance, label in test_data:
        bow_vec = make_bow_vector(instance, word_to_ix)
        log_pred = model(bow_vec)
        print("The log_pred is : ", log_pred)
        ix = torch.max(log_pred, dim=1)[1]
        print("The label is: ", ix_to_label[ix.item()])
    


{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Give': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}
Before training: 
The log_pred is :  tensor([[-0.6311, -0.7593]])
The label is:  SPANISH
The log_pred is :  tensor([[-1.0110, -0.4523]])
The label is:  ENGLISH
----------
Training...
epoch: 0, loss: 1.1032856702804565
epoch: 0, loss: 0.8261889219284058
epoch: 0, loss: 0.6373709440231323
epoch: 0, loss: 1.1923892498016357
epoch: 1, loss: 0.7105923891067505
epoch: 1, loss: 0.5002872943878174
epoch: 1, loss: 0.5397118330001831
epoch: 1, loss: 0.37703192234039307
epoch: 2, loss: 0.45892250537872314
epoch: 2, loss: 0.3762458860874176
epoch: 2, loss: 0.3831324279308319
epoch: 2, loss: 0.20928728580474854
epoch: 3, loss: 0.325185626745224
epoch: 3, loss: 0.30379316210746765
epoch: 3, loss: 0.2816413938999176
epoch: 3